# 中国語学習データベースをNeo4j(グラフデータベース)で構築してみる

以前、中国語学習を熱心にやっていたときに、単語や単文のデータベースを作っていたが、あまり長続きしなかった。理由は、正規化の部分での時間的コストが掛かること。項目を増やしたり、テーブルを追加したりなどなどの作業に追われて、あまり効率的ではなかったのが原因ではないかと考えている。そこで、グラフデータベースを使って面白いことが出来ないかを試してみることにした。

やりたいことを列挙すると、こんな感じ。
- 短文を登録すると、こんな構文を使っているねとか、この単語を使っているねというのが、ある程度自動でリンクされる。
- 特定に構文とか言い回しとかに沿った単文を例文として一覧で列挙する。
- 読み方などが分からない単語を登録する。
などなど。

## 構造を考えてみる

Neo4jには、各ノードをLabelで性格付けできるので、こんな感じのモノを作ってみるイメージ。
- 単語(Word)
- カテゴリ(Category)品詞とか
- 構文(Structure) 把構文みたいな
- 単文(Sentence)

## テキスト/CSVファイルからの取り込み

まずは単語の入ったcsvファイルを読み込む。項目数が多いので、DictReaderで辞書として取り込み、それをリスト化する

In [10]:
import csv
word_list = []
with open('chinesewords_20160219.csv', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        word_list.append(row)
print('{} words stored in word_list'.format(len(word_list)))

Wall time: 0 ns
5809 words stored in word_list


同じく単文の入ったtxtファイルを読み込む。こちらは項目数が今のところ1つだけなので、csvとして読む必要はない。ただし、行末の改行(\n)を消してやる必要あり。

In [18]:
sentence_list = []
with open('chinese_sentences20160218.txt', encoding='utf-8') as textfile:
    sentence_list = [row.rstrip() for row in textfile]
print('{} sentences stored in sentenc_list'.format(len(sentence_list)))

569 sentences stored in sentenc_list


改行が消えているかの確認。

In [4]:
sentence_list[-1]

'那么大的鱼，根本钓不到。'

## Neo4jとの接続

Neo4jとつないでみよう。あらかじめpy2neoをインストールしておく。
```
$pip install py2neo
```

In [3]:
%time
from py2neo import Graph, authenticate
from py2neo import Node, Relationship
authenticate('localhost:7474', 'neo4j', 'admin')
graph = Graph()    #http://localhost:7474

Wall time: 0 ns


(リカバリ用)すべてのノードと関係を削除する場合は、これを使う。

In [25]:
graph.delete_all()

CategoryとSentenceについては、単一の属性でユニークになるのでuniqueness_constraintを作っておく。

In [10]:
graph.schema.create_uniqueness_constraint("Category", "category")
graph.schema.create_uniqueness_constraint("Sentence", "sentence")

Wordについては、多音字を想定してkeywordとpinyinの組み合わせでユニークとする。複数項目の指定は出来ないので、エントリ追加時のロジックで対応する。

まずは短文登録を行う。重複ケースを考えてmerge_oneを使う。uniqueness_constraintとセットで使うとドキュメントに書いてある。

In [11]:
for sentence in sentence_list:
    graph.merge_one("Sentence", "sentence", sentence)

続いて単語の登録。ロジックを工夫しながらやる必要あり。

In [11]:
from datetime import datetime
start_time = datetime.now()
counter_a = 0
counter_c = 0
for word in word_list:
    counter_a += 1
    ws = list(graph.find("Word", "keyword", word['word']))
    found = False
    for w in ws:
        if w["pinyin"] == word['pinyin']:
            found = True
            break
    if found == True:
        new_word = w
    else:
        counter_c += 1
        new_word = Node("Word", keyword=word['word'], pinyin=word['pinyin'], pinyinf=word['pinyinf'], meaning=word['meaning'])
        graph.create(new_word)
    found = False
    if word.get('category'):
        cs = list(graph.find("Category", "category", word['category']))
        if len(cs) == 0:
            new_category = Node("Category", category=word['category'])
            graph.create(new_category)
        else:
            new_category = cs[0]
        word_is_categorized_as_category = Relationship(new_word, 'is_categorized_as', new_category)
        graph.create(word_is_categorized_as_category)
    
print("all count:{0}, created words:{1}".format(counter_a, counter_c))
endtime = datetime.now()
print ("It took {0} seconds to process".format{end_time - start_time})

Wall time: 0 ns
all count:5809, created words:5806


少し時間が掛かりすぎか？

## データベース内のデータのハンドリング(検索)

既存のデータの検索について、方法を考える。まずは、cypherを直接実行するexecuteを使うパターン。

In [29]:
w = graph.cypher.execute("MATCH (w:Word) WHERE w.keyword='熬夜' RETURN w") 
print(w)

   | w                                                                       
---+--------------------------------------------------------------------------
 1 | (n2:Word {keyword:"熬夜",meaning:"徹夜する",pinyin:"ao2 ye4",pinyinf:"áo yè"})



In [30]:
type(w)

py2neo.cypher.core.RecordList

RecordListからNordに変更する場合は、oneを使うことができる。ただし1件以上結果があった場合のみ。

In [35]:
w.one['keyword']

'熬夜'

もう少し保守的にいくなら、1件以上のノードが見つかったと仮定して、これをやってからOrder=1ならoneを呼ぶという感じだろうか。

In [38]:
w.to_subgraph().order

1

In [21]:
type(w[0])

py2neo.cypher.core.Record

速さは問題ないが、取り回しがよく分からない。続いては、findを使う方法。

In [25]:
w = list(graph.find("Word", "keyword", "熬夜"))

In [26]:
w

[<Node graph='http://localhost:7474/db/data/' ref='node/2' labels={'Word'} properties={'meaning': '徹夜する', 'pinyinf': 'áo yè', 'pinyin': 'ao2 ye4', 'keyword': '熬夜'}>]

In [27]:
w[0]["keyword"]

'熬夜'

これの方が取り回しは簡単な気がする。とりあえず汎用的に使えるように関数を書いてみる。

In [5]:
def search_word(keyword):
    ws = list(graph.find("Word", "keyword", keyword))
    if len(ws) == 0:
        print("Keyword {0} not found in database".format(keyword))
        return None
    else:
        print("Found {0} entries for keyword {1} in database".format(len(ws), keyword))
        return ws    

In [6]:
%time
word = search_word("菩萨")
print(word)

Wall time: 0 ns
Keyword 菩萨 not found in database
None


続いてピンイン検索も。pinyinfでの検索も考えた方が良いかもしれないが、仕組みは同じで簡単に作れるので、とりあえずpinyinだけで作る。

In [7]:
def search_pinyin(pinyin):
    ws = list(graph.find("Word", "pinyin", pinyin))
    if len(ws) == 0:
        print("No keywords found to pronounce like {0}".format(pinyin))
        return None
    else:
        print("Found {0} entries to pronounce like {1}".format(len(ws), pinyin))
        return ws

In [20]:
%time
words = search_pinyin('cheng2')
print(words)

Wall time: 0 ns
Found 5 entries to pronounce like cheng2
[<Node graph='http://localhost:7474/db/data/' ref='node/1532' labels={'Word'} properties={'meaning': '盛る', 'pinyinf': 'chéng', 'pinyin': 'cheng2', 'keyword': '盛'}>, <Node graph='http://localhost:7474/db/data/' ref='node/1533' labels={'Word'} properties={'meaning': '', 'pinyinf': 'chéng', 'pinyin': 'cheng2', 'keyword': '橙'}>, <Node graph='http://localhost:7474/db/data/' ref='node/1534' labels={'Word'} properties={'meaning': '', 'pinyinf': 'chéng', 'pinyin': 'cheng2', 'keyword': '乘'}>, <Node graph='http://localhost:7474/db/data/' ref='node/5631' labels={'Word'} properties={'meaning': '', 'pinyinf': 'chéng', 'pinyin': 'cheng2', 'keyword': '城'}>, <Node graph='http://localhost:7474/db/data/' ref='node/5736' labels={'Word'} properties={'meaning': '', 'pinyinf': 'chéng', 'pinyin': 'cheng2', 'keyword': '成'}>]


ひとまず大丈夫そう。

In [16]:
len(word_list)

5809

In [19]:
len(sentence_list)

569

Relationshipを使った検索も試しておく。

In [44]:
wl = graph.cypher.execute("MATCH (w:Word)-[r:is_categorized_as]->(c:Category) WHERE c.category='成語' RETURN w")

In [47]:
wl.to_subgraph().order

217

In [48]:
wl.to_subgraph()

<Subgraph order=217 size=0>

In [54]:
for w in wl:
    print(type(w), w)

<class 'py2neo.cypher.core.Record'>  w                                                                                                      
---------------------------------------------------------------------------------------------------------
 (n5558:Word {keyword:"总而言之",meaning:"要するに、つまり",pinyin:"zong3 er2 yan2 zhi1",pinyinf:"zǒng ér yán zhī"})

<class 'py2neo.cypher.core.Record'>  w                                                                                                          
-------------------------------------------------------------------------------------------------------------
 (n5544:Word {keyword:"自力更生",meaning:"人に頼らず自力で行う",pinyin:"zi4 li4 geng4 sheng1",pinyinf:"zì lì gèng shēng"})

<class 'py2neo.cypher.core.Record'>  w                                                                                                            
---------------------------------------------------------------------------------------------------------------
 (n5433:Word {keyword:"

Typeを見るとRecordとなっているので、取り回しは難しくなさそう。

引き続いて、STARTS WITHのテスト。これが使えると検索の幅が広がる。Neo4j側で動くモノであれば、基本的に大丈夫なはず。

In [55]:
sw = graph.cypher.execute("MATCH (w:Word) WHERE w.pinyin STARTS WITH ('shi2') return w")

In [56]:
sw

    | w                                                                                                               
----+------------------------------------------------------------------------------------------------------------------
  1 | (n737:Word {keyword:"石窟",meaning:"石窟",pinyin:"shi2 ku1",pinyinf:"shí kū"})                                      
  2 | (n738:Word {keyword:"实施",meaning:"実施する",pinyin:"shi2 shi1",pinyinf:"shí shī"})                                  
  3 | (n739:Word {keyword:"实事求是",meaning:"事実に即して問題を処理する",pinyin:"shi2 shi4 qiu2 shi4",pinyinf:"shí shì qiú shì "})    
  4 | (n740:Word {keyword:"拾金不昧",meaning:"金を拾っても着服しない、拾得物は届け出る",pinyin:"shi2 jin1 bu2 mei4",pinyinf:"shí jīn bú mèi"})
  5 | (n4026:Word {keyword:"拾",meaning:"",pinyin:"shi2",pinyinf:"shí"})                                               
  6 | (n4027:Word {keyword:"十",meaning:"",pinyin:"shi2",pinyinf:"shí"})                                               
  7 | (n4028:Word {keyword:"十分",meaning:"",piny

たとえば2文字の単語で、声調が1声+4声というような組み合わせが欲しい時は正規表現が必要。汎用性を考えると、上のパターンも含めて正規表現で良いかも。

In [62]:
regw = graph.cypher.execute("MATCH (w:Word) WHERE w.pinyin =~ '[a-z]*1 [a-z]*4' RETURN w")
regw.to_subgraph().order

470

単文の検索も試してみる。「不了」か「得了」を含む単文を検索してみる。

In [6]:
regs = graph.cypher.execute("MATCH (s:Sentence) WHERE s.sentence =~ '.*(不了|得了).*' RETURN s")
print("{} sentence(s) found in database".format(regs.to_subgraph().order))

32 sentence(s) found in database


エントリの属性の変更を行う。この例では意味(meaning)を変更している。

In [9]:
w = list(graph.find("Word", "keyword", "挨"))

In [10]:
print(w)

[<Node graph='http://localhost:7474/db/data/' ref='node/1110' labels={'Word'} properties={'keyword': '挨', 'pinyin': 'ai1', 'meaning': '', 'pinyinf': 'āi'}>]


In [12]:
w[0]['meaning'] = '近寄る，寄り添う，くっつく'
graph.push(w[0])

In [13]:
w = list(graph.find("Word", "keyword", "挨"))
print(w[0])

(n1110:Word {keyword:"挨",meaning:"近寄る，寄り添う，くっつく",pinyin:"ai1",pinyinf:"āi"})
